In [26]:
import os
import kfp

In [27]:
components_url = "/mnt/dkube/pipeline/components/"
dkube_training_op = kfp.components.load_component_from_file(
    components_url + "training/component.yaml"
)
dkube_serving_op = kfp.components.load_component_from_file(
    components_url + "serving/component.yaml"
)

token = os.getenv("DKUBE_USER_ACCESS_TOKEN")
client = kfp.Client(
    host=os.getenv("KF_PIPELINES_ENDPOINT"),
    existing_token=token,
    namespace=os.getenv("USERNAME"),
)
run_id = 0

In [28]:
@kfp.dsl.pipeline(
    name='fireweather',
    description='fireweather'
)

def fireweather_pipeline():

    # -------------------------------
    # TRAINING STEP
    # -------------------------------
    train = dkube_training_op(
        auth_token=token,
        container='{"image":"ocdr/d3-datascience-sklearn:v0.23.2-1"}',
        framework="sklearn",
        version="0.23.2",
        program="project2",                  # code repo
        run_script="python training.py",        # script
        datasets='["dataset2"]',          # dataset name
        input_dataset_mounts='["/mnt/ou/"]',    # Will mount dataset at /mnt/ou
        outputs='["fireweather"]',             # model name
        output_mounts='["/mnt/model"]',          # Save model path
        envs='[]'
    )

    # -------------------------------
    # SERVING STEP
    # -------------------------------
    serving = dkube_serving_op(
        auth_token=token,
        model=train.outputs["artifact"],     # Model produced by training
        device="cpu",
        serving_image='{"image":"ocdr/sklearnserver:0.23.2"}',
    ).after(train)


In [29]:
client.create_run_from_pipeline_func(fireweather_pipeline, arguments={})

RunPipelineResult(run_id=53311653-da92-4680-a7a2-9e91d21216fd)